In [6]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain.llms.base import LLM
import g4f

In [8]:
from langchain.llms.base import LLM
import g4f
from typing import Optional, List


class G4FLLM(LLM):
    model_name: str = "gpt_3_5_turbo"  # Define your model here, default is gpt_3_5_turbo from g4f

    # This method is responsible for making the API call to g4f's model
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        try:
            # Use g4f to generate the completion based on the prompt
            response = g4f.ChatCompletion.create(
                model=g4f.models.gpt_3_5_turbo,
                messages=[{"role": "user", "content": prompt}]
            )
            # Extract the generated content from the response
            return response["choices"][0]["message"]["content"]
        except Exception as e:
            return f"Error generating response: {str(e)}"

    # Required method for langchain to get identifying parameters for the model
    @property
    def _identifying_params(self) -> dict:
        return {"model_name": self.model_name}

    # Correct method to identify the type of LLM
    def _llm_type(self) -> str:
        return "g4f"

In [5]:
myllm=G4FLLM()

In [9]:
import json
import g4f

# Simulated callback class (to mimic get_openai_callback())
class CustomCallbackHandler:
    def __init__(self):
        self.tokens = 0
        self.cost = 0
        self.completions = []

    # Allow the class to be used with "with" statements
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        pass

    def on_start(self, data):
        print(f"Processing: {data}")

    def on_new_token(self, token):
        self.tokens += 1
        print(f"New token: {token}")

    def on_end(self, response):
        self.completions.append(response)
        print(f"Completed response: {response}")

# Simulating the callback context manager
def get_g4f_callback():
    return CustomCallbackHandler()

# Function to generate MCQs from the input text
def generate_mcqs_from_text(input_text, callback):
    callback.on_start(input_text)

    try:
        # Call g4f API to generate MCQs
        response = g4f.ChatCompletion.create(
            model=g4f.models.default,  # Replace with the correct model
            messages=[
                {"role": "system", "content": f"Generate MCQs from the following text: {input_text}"}
            ]
        )

        # Debugging: Print the raw response
        print("Raw response:", response)

        # Check if response is a dictionary
        if isinstance(response, dict):
            quiz = response.get("quiz")
            if quiz is not None:
                print("Quiz data found.")
                
                # Process quiz data
                quiz_table_data = []
                for key, value in quiz.items():
                    mcq = value["mcq"]
                    options = " | ".join(
                        [
                            f"{option}: {option_value}"
                            for option, option_value in value["options"].items()
                        ]
                    )
                    correct = value["correct"]
                    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
                
                # Output the quiz table data
                for quiz_item in quiz_table_data:
                    print(quiz_item)
            else:
                print("Quiz key not found in response.")
        else:
            raise TypeError("Unexpected response type")

        callback.on_end(response)
        return response

    except Exception as e:
        print("An error occurred:", str(e))
        return None

# Function to read text from a file
def read_text_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except Exception as e:
        print(f"Error reading file: {str(e)}")
        return None

# Main logic
if __name__ == "__main__":
    # Read the content of data.txt
    text_content = read_text_from_file('data.txt')
    
    if text_content:
        # Using the callback structure with g4f
        with get_g4f_callback() as cb:
            # Generate MCQs from the text content
            response = generate_mcqs_from_text(text_content, cb)
    else:
        print("No content found in data.txt.")


Processing: 
Generative artificial intelligence (AI) represents one of the most transformative advancements in modern technology, with the potential to reshape industries, redefine creativity, and automate tasks that were once considered strictly human. At its core, generative AI refers to systems capable of creating novel content, including text, images, music, videos, and even entire virtual worlds. Unlike traditional AI, which primarily focuses on classification, prediction, and decision-making tasks, generative AI is designed to mimic creativity by producing new data based on the patterns and structures learned from existing datasets.

The concept of generative AI is rooted in several key technologies. One of the most influential is the Generative Adversarial Network (GAN), introduced by Ian Goodfellow in 2014. GANs are made up of two neural networks: a generator, which creates data that mimics the target dataset, and a discriminator, which evaluates whether the generated data is r